#Clone Repository

In [14]:
!rm -rf *
!rm -rf .git* .config
!ls -la

total 12
drwxr-xr-x 1 root root 4096 Jun 23 19:28 .
drwxr-xr-x 1 root root 4096 Jun 23 08:40 ..


In [15]:
!git clone https://github.com/manuhg/yolo_retrain.git .
# !git pull

Cloning into '.'...
remote: Enumerating objects: 94, done.
remote: Counting objects: 100% (94/94), done.
remote: Compressing objects: 100% (65/65), done.
remote: Total 94 (delta 48), reused 73 (delta 27), pack-reused 0
Unpacking objects: 100% (94/94), done.


# Modi Rahul

In [1]:
from train import train, run_inference

pip -q install natsort 


In [0]:
import os

def exec_cmd(cmdstr):
    os.popen(cmdstr).read()
    
def ds_handle():
    exec_cmd('unzip data.zip')

model_name = 'yolov2'

In [3]:
train('.',model_name,dataset_dw_func=ds_handle)

Cloning darknet repository
git clone https://github.com/AlexeyAB/darknet.git dkn 
mv -v dkn/* ./ renamed 'dkn/3rdparty' -> './3rdparty'
renamed 'dkn/CMakeLists.txt' -> './CMakeLists.txt'
renamed 'dkn/DarknetConfig.cmake.in' -> './DarknetConfig.cmake.in'
renamed 'dkn/LICENSE' -> './LICENSE'
renamed 'dkn/Makefile' -> './Makefile'
renamed 'dkn/README.md' -> './README.md'
renamed 'dkn/appveyor.yml' -> './appveyor.yml'
renamed 'dkn/build' -> './build'
renamed 'dkn/build.ps1' -> './build.ps1'
renamed 'dkn/build.sh' -> './build.sh'
renamed 'dkn/cfg' -> './cfg'
renamed 'dkn/cmake' -> './cmake'
renamed 'dkn/darknet.py' -> './darknet.py'
renamed 'dkn/darknet_video.py' -> './darknet_video.py'
renamed 'dkn/data' -> './data'
renamed 'dkn/image_yolov2.sh' -> './image_yolov2.sh'
renamed 'dkn/image_yolov3.sh' -> './image_yolov3.sh'
renamed 'dkn/include' -> './include'
renamed 'dkn/json_mjpeg_streams.sh' -> './json_mjpeg_streams.sh'
renamed 'dkn/net_cam_v3.sh' -> './net_cam_v3.sh'
renamed 'dkn/scripts'

KeyboardInterrupt: ignored

In [13]:
!./darknet detector test cfg/obj.data cfg/yolo_custom.cfg backup/yolo_custom_last.weights data_dir/images/modi_00.jpg

   layer   filters  size/strd(dil)      input                output
   0 conv     32      3 x 3/ 1    608 x 608 x   3  ->  608 x 608 x  32 0.639 BF
   1 max              2 x 2/ 2    608 x 608 x  32 ->  304 x 304 x  32 0.012 BF
   2 conv     64      3 x 3/ 1    304 x 304 x  32  ->  304 x 304 x  64 3.407 BF
   3 max              2 x 2/ 2    304 x 304 x  64 ->  152 x 152 x  64 0.006 BF
   4 conv    128      3 x 3/ 1    152 x 152 x  64  ->  152 x 152 x 128 3.407 BF
   5 conv     64      1 x 1/ 1    152 x 152 x 128  ->  152 x 152 x  64 0.379 BF
   6 conv    128      3 x 3/ 1    152 x 152 x  64  ->  152 x 152 x 128 3.407 BF
   7 max              2 x 2/ 2    152 x 152 x 128 ->   76 x  76 x 128 0.003 BF
   8 conv    256      3 x 3/ 1     76 x  76 x 128  ->   76 x  76 x 256 3.407 BF
   9 conv    128      1 x 1/ 1     76 x  76 x 256  ->   76 x  76 x 128 0.379 BF
  10 conv    256      3 x 3/ 1     76 x  76 x 128  ->   76 x  76 x 256 3.407 BF
  11 max              2 x 2/ 2     76 x  76 x 256 ->   

In [0]:
!cp cfg/obj.names ./

# Do it at once

In [17]:
!python train.py -d . -ts Pascal_VOC -m yolov2-tiny

pip -q install natsort 
--2019-06-23 18:42:29--  https://pjreddie.com/media/files/VOCtrainval_11-May-2012.tar
Resolving pjreddie.com (pjreddie.com)... 128.208.4.108
Connecting to pjreddie.com (pjreddie.com)|128.208.4.108|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1999639040 (1.9G) [application/octet-stream]
Saving to: ‘VOCtrainval_11-May-2012.tar’

 VOCtrainval_11-May  47%[========>           ] 901.77M  30.1MB/s    eta 35s    


 # Do it step by step

## Get  Pascal VOC dataset

In [4]:
!wget https://pjreddie.com/media/files/VOCtrainval_11-May-2012.tar
!wget https://pjreddie.com/media/files/VOCtrainval_06-Nov-2007.tar
!wget https://pjreddie.com/media/files/VOCtest_06-Nov-2007.tar
!tar xf VOCtrainval_11-May-2012.tar
!tar xf VOCtrainval_06-Nov-2007.tar
!tar xf VOCtest_06-Nov-2007.tar

!wget https://pjreddie.com/media/files/voc_label.py
!python voc_label.py

!cat 2007_train.txt 2012_*.txt > train.txt

!cp 2007_val.txt test.txt

!cp data/voc.names classes.txt

!mkdir bkup
!mv 2* bkup/
!mv *.tar bkup/

--2019-06-23 09:04:55--  https://pjreddie.com/media/files/VOCtrainval_11-May-2012.tar
Resolving pjreddie.com (pjreddie.com)... 128.208.4.108
Connecting to pjreddie.com (pjreddie.com)|128.208.4.108|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1999639040 (1.9G) [application/octet-stream]
Saving to: ‘VOCtrainval_11-May-2012.tar.1’

l_11-May-2012.tar.1   5%[>                   ] 113.80M  29.0MB/s    eta 70s    

KeyboardInterrupt: ignored

## Get NFPA dataset

In [9]:
!pip install gdown

In [10]:
!gdown https://drive.google.com/uc?id=1_gJcci9p6cS0EucumFt2gpwA0tqgP7_f&export=download

Downloading...
From: https://drive.google.com/uc?id=1_gJcci9p6cS0EucumFt2gpwA0tqgP7_f
To: /content/NFPA_dataset.zip
20.4MB [00:00, 44.0MB/s]


In [24]:
!unzip NFPA_dataset.zip

Archive:  NFPA_dataset.zip
   creating: NFPA dataset/
  inflating: NFPA dataset/pos-1.jpg  
  inflating: NFPA dataset/pos-1.txt  
  inflating: NFPA dataset/pos-100.jpg  
  inflating: NFPA dataset/pos-100.txt  
  inflating: NFPA dataset/pos-101.jpg  
  inflating: NFPA dataset/pos-101.txt  
  inflating: NFPA dataset/pos-102.jpg  
  inflating: NFPA dataset/pos-102.txt  
  inflating: NFPA dataset/pos-103.jpg  
  inflating: NFPA dataset/pos-103.txt  
  inflating: NFPA dataset/pos-104.jpg  
  inflating: NFPA dataset/pos-104.txt  
  inflating: NFPA dataset/pos-105.jpg  
  inflating: NFPA dataset/pos-105.txt  
  inflating: NFPA dataset/pos-106.jpg  
  inflating: NFPA dataset/pos-106.txt  
  inflating: NFPA dataset/pos-107.jpg  
  inflating: NFPA dataset/pos-107.txt  
  inflating: NFPA dataset/pos-108.jpg  
  inflating: NFPA dataset/pos-108.txt  
  inflating: NFPA dataset/pos-109.jpg  
  inflating: NFPA dataset/pos-109.txt  
  inflating: NFPA dataset/pos-110.jpg  
  inflating: NFPA dataset/pos-

In [23]:
!mkdir data

mkdir: cannot create directory ‘data’: File exists


In [0]:
!mv 'NFPA dataset' data/nfpa

In [0]:
!cp nfpa/data/train.txt ./
!cp nfpa/data/test.txt ./
!printf "NFPA" > classes.txt

## Train

### Prepare config

In [0]:
data_dir = '.'
class_names_file = 'classes.txt'
batch_size, subdivisions = int('64'), int('8')
model_name='yolov2'
model_url='https://pjreddie.com/media/files/darknet19_448.conv.23'
filename='yolo_custom.cfg'

import os
import sys
import subprocess
from gen_files import gen
import argparse

def exec_cmd(cmdstr):
    print(cmdstr,os.popen(cmdstr).read())

In [41]:
if not os.path.isfile('./darknet'):
    print('Cloning darknet repository')
    exec_cmd('git clone https://github.com/pjreddie/darknet.git dkn')
    exec_cmd('mv -v dkn/* ./')
    exec_cmd("sed -i 's/GPU=0/GPU=1/;s/CUDNN=0/CUDNN=1/;s/OPENCV=0/OPENCV=1/;s/OPENMP=0/OPENMP=1/;' Makefile")
    exec_cmd('make -j8')

if not os.path.isfile('darknet19_448.conv.23'):
    print('Downloading darknet model')
    exec_cmd('wget '+model_url)

Cloning darknet repository
git clone https://github.com/pjreddie/darknet.git dkn 
mv -v dkn/* ./ renamed 'dkn/LICENSE' -> './LICENSE'
renamed 'dkn/LICENSE.fuck' -> './LICENSE.fuck'
renamed 'dkn/LICENSE.gen' -> './LICENSE.gen'
renamed 'dkn/LICENSE.gpl' -> './LICENSE.gpl'
renamed 'dkn/LICENSE.meta' -> './LICENSE.meta'
renamed 'dkn/LICENSE.mit' -> './LICENSE.mit'
renamed 'dkn/LICENSE.v1' -> './LICENSE.v1'
renamed 'dkn/Makefile' -> './Makefile'
renamed 'dkn/README.md' -> './README.md'
renamed 'dkn/cfg' -> './cfg'
renamed 'dkn/data' -> './data'
renamed 'dkn/examples' -> './examples'
renamed 'dkn/include' -> './include'
renamed 'dkn/python' -> './python'
renamed 'dkn/scripts' -> './scripts'
renamed 'dkn/src' -> './src'

sed -i 's/GPU=0/GPU=1/;s/CUDNN=0/CUDNN=1/;s/OPENCV=0/OPENCV=1/;s/OPENMP=0/OPENMP=1/;' Makefile 
make -j8 mkdir -p obj
mkdir -p backup
mkdir -p results
gcc -Iinclude/ -Isrc/ -DOPENCV `pkg-config --cflags opencv`  -DGPU -I/usr/local/cuda/include/ -DCUDNN  -Wall -Wno-unused-resu

In [28]:
with open(data_dir+'/'+class_names_file) as f:
            class_names = list(
                map(lambda s: s.replace('\n', '').strip(), f.readlines()))
            class_names = list(filter(None,class_names))

data_file, names_file, cfg_file = gen(
    class_names, model_name=model_name, batch_size=batch_size, subdivisions=subdivisions, filename=filename)
flag = True
print(data_file, names_file, cfg_file)

Configuring to train  yolov2  classes: 1
cp cfgs/yolov2.cfg cfg/yolo_custom.cfg 
sed -i '6s/.*/batch=64/;7s/.*/subdivisions=8/;237s/.*/filters=30/;244s/.*/classes=1/;' 'cfg/yolo_custom.cfg' 
cfg/obj.data cfg/obj.names cfg/yolo_custom.cfg


###  Train the model

#### sub process

In [0]:
print('Traning the model')
cmd = './darknet detector train '+data_file +' '+cfg_file+' darknet19_448.conv.23'
p = subprocess.Popen(cmd, stdout=subprocess.PIPE, bufsize=1,shell = True)
for line in iter(p.stdout.readline,''):
    print(line)
p.stdout.close()
p.wait()

#### Direct commandline

In [18]:
!./darknet detector train cfg/obj.data cfg/yolo_custom.cfg darknet19_448.conv.23 -dont_show

yolo_custom
   layer   filters  size/strd(dil)      input                output
   0 conv     32      3 x 3/ 1    608 x 608 x   3  ->  608 x 608 x  32 0.639 BF
   1 max              2 x 2/ 2    608 x 608 x  32 ->  304 x 304 x  32 0.012 BF
   2 conv     64      3 x 3/ 1    304 x 304 x  32  ->  304 x 304 x  64 3.407 BF
   3 max              2 x 2/ 2    304 x 304 x  64 ->  152 x 152 x  64 0.006 BF
   4 conv    128      3 x 3/ 1    152 x 152 x  64  ->  152 x 152 x 128 3.407 BF
   5 conv     64      1 x 1/ 1    152 x 152 x 128  ->  152 x 152 x  64 0.379 BF
   6 conv    128      3 x 3/ 1    152 x 152 x  64  ->  152 x 152 x 128 3.407 BF
   7 max              2 x 2/ 2    152 x 152 x 128 ->   76 x  76 x 128 0.003 BF
   8 conv    256      3 x 3/ 1     76 x  76 x 128  ->   76 x  76 x 256 3.407 BF
   9 conv    128      1 x 1/ 1     76 x  76 x 256  ->   76 x  76 x 128 0.379 BF
  10 conv    256      3 x 3/ 1     76 x  76 x 128  ->   76 x  76 x 256 3.407 BF
  11 max              2 x 2/ 2     76 x  76